# Data Cleaning and Preperation

In [1]:
import os
import utils
from mysklearn import myutils 
from mysklearn.mypytable import MyPyTable

In [2]:

fpath = os.path.join("Data", "beer_reviews.csv")
table = MyPyTable().load_from_file(fpath)

print("loaded")

table.drop_column("review_taste")
table.drop_column("review_palate")
table.drop_column("review_profilename")
table.drop_column("review_aroma")
table.drop_column("review_time")
table.drop_column("review_appearance")

groups = table.group_by("beer_beerid")

print("grouped")

cleaned_table = MyPyTable()
cleaned_table.column_names = table.column_names
cleaned_table.column_names.remove("review_overall")
cleaned_table.column_names.append("rating")


loaded
grouped


In [3]:
for group in groups.values():
    col = group.get_column("review_overall")
    average = sum(col) / len(col)
    val = utils.high_low_discretizer(average)

    group.drop_column("review_overall")

    row = group.data[0]
    row.append(val)
    cleaned_table.data.append(row)

In [4]:
# remove NaN
cleaned_table.remove_rows_with_missing_values()

### Cleaning ABV Values
We need to discretize the abv values because there are too many different values. Originally, they are doubles. Most of them have one decimal place, although some have more. We chose to discretize the abv by rounding down to the nearest integer. This will limit the number of different values to a reasonable amount. We also need to restrict the upper limit of the abv value, as some of the values above 18 only have a couple instances. We chose to solve this issue by taking all of the values above 18 and turning them into 18, because this is the cutoff where the number of instances for each value falls off. This is done in the following code.

In [5]:
# round abv
col_index = cleaned_table.get_column_index("beer_abv")
for row in cleaned_table.data:
    new_value = int(row[col_index])
    if new_value >= 18:
        new_value = 18
    row[col_index] = new_value



In [6]:
cleaned_table.save_to_file("Data/beer_reviews_cleaned.csv")

In [7]:
mt = MyPyTable()
mt.load_from_file("Data/beer_reviews_cleaned.csv")

In [8]:
mt.column_names
types = mt.get_column("beer_style")
uniqueNames = []
for name in types:
    if not name in uniqueNames:
        uniqueNames.append(name)
NameSiplifier = {}
count = 0
for name in uniqueNames:
    if "Pilsener" in name or "Pilsner" in name:
        NameSiplifier[name] = "Pilsener"
        uniqueNames[uniqueNames.index(name)] = "Pilsener"

for name in uniqueNames:
    if "Stout" in name or "stout" in name:
        NameSiplifier[name] = "Stout"
        uniqueNames[uniqueNames.index(name)] = "Stout"

for name in uniqueNames:
    if "IPA" in name:
        NameSiplifier[name] = "IPA"
        uniqueNames[uniqueNames.index(name)] = "IPA"

for name in uniqueNames:
    if "Porter" in name:
        NameSiplifier[name] = "Porter"
        uniqueNames[uniqueNames.index(name)] = "Porter"
for name in uniqueNames:
    if "Lager" in name:
        NameSiplifier[name] = "Lager"
        uniqueNames[uniqueNames.index(name)] = "Lager"        
temp2 = []
for name in uniqueNames:
    if "Strong" in name:
        count = count + 1
        temp2.append(name)
        NameSiplifier[name] = "Strong Ale"
        uniqueNames[uniqueNames.index(name)] = "Strong Al"
for name in uniqueNames:
    if "Dark" in name:
        count = count + 1
        #temp2.append(name)
        NameSiplifier[name] = "Dark Ale"
        uniqueNames[uniqueNames.index(name)] = "Dark Al"
for name in uniqueNames:
    if "Pale" in name:
        count = count + 1
        #temp2.append(name)
        NameSiplifier[name] = "Dark Ale"
        uniqueNames[uniqueNames.index(name)] = "Dark Al"
for name in uniqueNames:
    if "Ale" in name:
        count = count + 1
        temp2.append(name)
        NameSiplifier[name] = "Ale"
        uniqueNames[uniqueNames.index(name)] = "Ale" 
others = ["Faro","Bière de Champagne","Sahti","Gose","Kvass","Braggot","Eisbock","Roggenbier","Lambic","Happoshu",
    "Black","Gueuze","Weissbier","Wheatwine","Kristalweizen","Chile Beer","Quadrupel"]
for name in uniqueNames:
    for other in others:
        if  other in name:
            count = count + 1
            temp2.append(name)
            NameSiplifier[name] = "Other"
            uniqueNames[uniqueNames.index(name)] = "Other"    

#print(count)

types = mt.get_column("beer_style")




    



Time to updata the data file so that ther will be less beer types 

In [9]:
types = mt.get_column("beer_style")
for i,type in enumerate(types):
    try:
        types[i] = NameSiplifier[type]
    except:
        1+1
    
mt.drop_column("beer_style")
mt.add_column("beer_style",types,index=3)
types = mt.get_column("beer_style")

mt.save_to_file("Data/beer_reviews_cleaned.csv")